# Estudo Algoritmos de Regressão Polinomial e Regressão Linear.
- O objetivo deste notebook é o estudo dos algoritmos de Regressão Polinomial e Regressão Linear do scikit-learn.
- Vamos utilizar nesse estudo o dataset 'house-prices.csv'.
- Iremos fazer uma breve comparação entre os resultados destes dois algoritmos para esta base dados.
- Este dataset contém 21 atributos e 21.613 instâncias (linhas). 

  

In [1]:
# Importando as bibliotecas iniciais
import pandas as pd
import numpy as np

In [2]:
# Importando o dataset
base = pd.read_csv('./house-prices.csv')

In [3]:
# Verificando o shape do dataset
base.shape

(21613, 21)

In [4]:
# Visualiza as primeira linhas
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [5]:
# Verificando os atributos núméricos.
base.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


# Pré-processamento - Tratamento dos dados

In [6]:
# Seleciona os atributos previsores
# Não vamos considerar as colunas "id" e "date". A coluna "price" será o valor que vamos tentar prever. 
previsores = base.iloc[:, 3:19].values
previsores

array([[ 3.00000e+00,  1.00000e+00,  1.18000e+03, ...,  9.81780e+04,
         4.75112e+01, -1.22257e+02],
       [ 3.00000e+00,  2.25000e+00,  2.57000e+03, ...,  9.81250e+04,
         4.77210e+01, -1.22319e+02],
       [ 2.00000e+00,  1.00000e+00,  7.70000e+02, ...,  9.80280e+04,
         4.77379e+01, -1.22233e+02],
       ...,
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75944e+01, -1.22299e+02],
       [ 3.00000e+00,  2.50000e+00,  1.60000e+03, ...,  9.80270e+04,
         4.75345e+01, -1.22069e+02],
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75941e+01, -1.22299e+02]])

In [7]:
# Seleciona a coluna o alvo (target)
# Nesse caso será a coluna "price"
preco_alvo = base.iloc[:,2].values
preco_alvo

array([221900., 538000., 180000., ..., 402101., 400000., 325000.])

In [8]:
# StandardScaler para colocar todos os valores na mesma escala (escalonamento).
# Teste do escalonamente de todos atributos, tivemos melhoria no modelo. Tivemos Acurácia de 80.46% a mesma da padronização parcial. Por isso, vamos deixar ele dasabilitado.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [9]:
# Separa os dados pata treinamento e teste
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, preco_alvo_treinamento, preco_alvo_teste = train_test_split(previsores, preco_alvo , test_size = 0.25, random_state = 0)


In [10]:
previsores_treinamento.shape

(16209, 16)

In [11]:
previsores_teste.shape

(5404, 16)

In [12]:
preco_alvo_teste.shape

(5404,)

# Criando os Modelos de Previsão para comparação

# Criando o modelo de Regressão linear simples

In [13]:
# Cria o modelo de previsão LinearRegression
from sklearn.linear_model import LinearRegression
regressor_linear = LinearRegression()
regressor_linear.fit(previsores_treinamento, preco_alvo_treinamento)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Criando o modelo de Regressão polinomial

In [14]:
# Preparando as configurações para a criação do modelo de previsão Polynomial

from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree = 3)
previsores_treinamento_poly = poly.fit_transform(previsores_treinamento)
previsores_teste_poly = poly.transform(previsores_teste)

In [15]:
# Cria o modelo de previsão Polynomial

from sklearn.linear_model import LinearRegression
regressor_poly = LinearRegression()
regressor_poly .fit(previsores_treinamento_poly, preco_alvo_treinamento)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Verificando a performance dos Modelos

In [16]:
# Score do Modelo Linea
score_linear = regressor_linear.score(previsores_treinamento, preco_alvo_treinamento)
print('Score do Modelo Linear: %.4f ' % score_linear)

Score do Modelo Linear: 0.7012 


In [17]:
# Score do Modelo Polinomial
score_poly = regressor_poly.score(previsores_treinamento_poly, preco_alvo_treinamento)
print('Score do Modelo Polinomial: %.4f ' % score_poly)

Score do Modelo Polinomial: 0.8903 


In [18]:
# Realizando o testes do modelo Linear
previsao_linear = regressor_linear.predict(previsores_teste)
previsao_linear

array([ 383618.08720934, 1521323.22249092,  548807.65183475, ...,
        783051.88887858,  245377.56548516,  595962.47157474])

In [19]:
# Realizando o testes do modelo Polinomial
previsao_poly = regressor_poly.predict(previsores_teste_poly)
previsao_poly

array([ 337042.44140625, 2090840.00390625,  593830.25390625, ...,
        648522.12890625,  435498.41015625,  587169.19140625])

In [20]:
# Analisando a performance dos Modelos

from sklearn.metrics import mean_absolute_error

mae_linear = mean_absolute_error(preco_alvo_teste, previsao_linear)
mae_poly = mean_absolute_error(preco_alvo_teste, previsao_poly)

print('\nMean Absolute Error do modelo Linear: %.2f ' % mae_linear)
print('\nMean Absolute Error do modelo Polinomial: %.2f ' % mae_poly)


Mean Absolute Error do modelo Linear: 123470.23 

Mean Absolute Error do modelo Polinomial: 92496.30 


# Conclusão:

- Podemos observar nos resultados acima que o 'Algoritmo Decision Tree' obteve um melhor resultado em relação ao 'Algoritmo Naive Bayes' para o dataset 'census.csv' quando utilizado os mesmos pré-processamentos .